In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
import os
import re
import bs4
import time
import random
import urllib
import argparse
import itertools
import urllib.request
import multiprocessing
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool
 

img_count = 0
downloaded_count = 0


def get_paintings(count, genre):
    try:
        time.sleep(2.0 * random.random())
        url = f"https://www.wikiart.org/en/paintings-by-genre/{genre}/{str(count)}"
        print(url)
        soup = BeautifulSoup(urllib.request.urlopen(url), "lxml")
        regex = r'https?://uploads[0-9]+[^/\s]+/\S+\.jpg'
        url_list = re.findall(regex, str(soup.html()))
        count += len(url_list)
        return url_list
    except:
        print('failed')


def downloader(link, genre, output_dir):
    global downloaded_count, img_count
    item, file = link
    filepath = file.split('/')
    savepath = '%s/%s/%s' % (output_dir, genre, filepath[-1])    
    try:
        time.sleep(0.2) 
        urllib.request.urlretrieve(file, savepath)
        downloaded_count += 1
        if downloaded_count % 100 == 0:
            print('downloaded %d / %d...' % (downloaded_count, img_count))
    except Exception as e:
        print("failed " + str(file), e) 


def main(genre, num_pages, output_dir):
    global img_count
    print('gathering links')
    threadpool = Pool(multiprocessing.cpu_count() - 1)
    numbers = list(range(1, num_pages))
    wikiart_pages = threadpool.starmap(get_paintings, zip(numbers, itertools.repeat(genre))) 
    threadpool.close()
    threadpool.join()

    pages = [page for page in wikiart_pages if page ]
    items = [item for sublist in pages for item in sublist]
    items = list(set(items))
    img_count = len(items)
    
    if not os.path.isdir('%s/%s' % (output_dir, genre)):
        os.mkdir('%s/%s' % (output_dir, genre))
    
    print('attempting to download %d images' % img_count)
    threadpool = Pool(multiprocessing.cpu_count() - 1)
    threadpool.starmap(downloader, zip(enumerate(items), itertools.repeat(genre), itertools.repeat(output_dir)))
    threadpool.close    
    threadpool.close()



main('portrait', 1000, 'drive/MyDrive/output_dir')

In [ ]:
import os
import numpy as np
from PIL import Image
from keras.optimizers import Adam
from IPython.display import Image as Iimage
from keras.utils.vis_utils import plot_model
from keras.models import Sequential, Model, load_model
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D

In [ ]:
MARGIN = 4
ROWS = 4
COLS = 7

NOISE_SIZE = 100
SAVE_FREQ = 100

EPOCHS = 100000
GENERATE_RES = 3

BATCH_SIZE = 128
IMAGE_SIZE = 128
IMAGE_CHANNELS = 3

IMAGE_DIR = 'drive/MyDrive/output_dir/portrait'

images_path = IMAGE_DIR 

In [ ]:
img = os.listdir(images_path)
idx = np.random.randint(0, len(img), BATCH_SIZE)

for i in range(len(img)):
    path = os.path.join(images_path, img[i])
    print(path)
    image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS).convert('RGB')
    image.save(path)


In [ ]:
def discriminator(image_shape):
    disc = Sequential()
    
    disc.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))

    disc.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(BatchNormalization(momentum=0.8))
    disc.add(Dropout(0.25))

    disc.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(BatchNormalization(momentum=0.8))

    disc.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(BatchNormalization(momentum=0.8))
    disc.add(Dropout(0.25))

    disc.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    disc.add(BatchNormalization(momentum=0.8))
    disc.add(LeakyReLU(alpha=0.2))

    disc.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(BatchNormalization(momentum=0.8))
    disc.add(Dropout(0.25))


    disc.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(BatchNormalization(momentum=0.8))

    disc.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    disc.add(BatchNormalization(momentum=0.8))
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(Dropout(0.25))

    disc.add(Flatten())
    disc.add(LeakyReLU(alpha=0.2))
    disc.add(Dense(1, activation="sigmoid"))
    
    input_image = Input(shape=image_shape)
    validity = disc(input_image)

    plot_model(disc, show_shapes=True, show_layer_names=True, to_file='discriminator.png')

    return Model(input_image, validity)



In [ ]:
# discriminator((IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
# Iimage('discriminator.png')

In [ ]:
def generator(noise_size, channels):
    gen = Sequential()
    gen.add(Dense(4 * 4 * 256, activation="relu", input_dim=noise_size))
    gen.add(Reshape((4, 4, 256)))
    gen.add(UpSampling2D())
    
    gen.add(Conv2D(256, kernel_size=3, padding="same"))
    gen.add(BatchNormalization(momentum=0.8))
    gen.add(Activation("relu"))
    gen.add(UpSampling2D())

    gen.add(Conv2D(256, kernel_size=3, padding="same"))
    gen.add(BatchNormalization(momentum=0.8))
    gen.add(Activation("relu"))

    for i in range(GENERATE_RES):
         gen.add(UpSampling2D())
         gen.add(Conv2D(256, kernel_size=3, padding="same"))
         gen.add(BatchNormalization(momentum=0.8))
         gen.add(Activation("relu"))
         gen.summary()

    gen.add(Conv2D(channels, kernel_size=3, padding="same"))
    gen.add(Activation("tanh"))
    
    input = Input(shape=(noise_size,))
    generated_image = gen(input)
    
    # plot_model(gen, show_shapes=True, show_layer_names=True, to_file='generator.png')

    return Model(input, generated_image)

In [ ]:
# build_generator(NOISE_SIZE, IMAGE_CHANNELS)
# Iimage('generator.png')

In [ ]:
def save_img(cnt, noise):
    image_array = np.full((
        MARGIN + (ROWS * (IMAGE_SIZE + MARGIN)),
        MARGIN + (COLS * (IMAGE_SIZE + MARGIN)), 3),
        255, dtype=np.uint8)
    
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0

    for row in range(ROWS):
        for col in range(COLS):
            r = row * (IMAGE_SIZE + MARGIN) + MARGIN
            c = col * (IMAGE_SIZE + MARGIN) + MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
            
    output_path = 'output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [ ]:
img = os.listdir(images_path)

shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = Adam(1.5e-9, 0.5)
discriminator = discriminator(shape)

discriminator.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=["accuracy"])
generator = generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False

validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
fixed_noise = np.random.normal(0, 1, (ROWS * COLS, NOISE_SIZE))

cnt = 1

for epoch in range(EPOCHS):
  training_data = []
  idx = np.random.randint(0, len(img), BATCH_SIZE)

  for i in idx:
      path = os.path.join(images_path, img[i])
      image = Image.open(path)
      training_data.append(np.asarray(image))
  
  training_data = np.reshape(training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
  x_real = training_data / 127.5 - 1

 
  noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
  x_fake = generator.predict(noise)
 
  discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
  discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)
  
  discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
  generator_metric = combined.train_on_batch(noise, y_real)

  if epoch % SAVE_FREQ == 0:
    save_img(cnt, fixed_noise)
    cnt += 1
  
    print(f"{epoch} epoch, Generator accuracy: {100 * generator_metric[1]}, Discriminator accuracy: {100 *  discriminator_metric[1]}")


In [ ]:
  # !rm -rf output